In [4]:
#packages
import pandas as pd
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import metrics
import matplotlib.pyplot as plt
# With missforest we can use random forest to impute data. This is better than mean by far
import sys
import sklearn.neighbors._base
from pandas_profiling import ProfileReport
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

In [5]:
# loading data
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')
train_df.head()
test_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [12]:
print(len(train_df))
print(len(test_df))

1460
1459


In [ ]:
# removing columns with a high ratio of NAs
def remove_high_nan(df, ratio = 0.65):
    for i in df.columns:
        if df[i].isnull().mean() > ratio:
            df.drop(i,axis = 1, inplace = True)


remove_high_nan(data)
data_2 = data.copy()
data.isnull().sum().plot(kind = 'bar', fontsize = 12, figsize = (12,4))
plt. title('Null values we Have to handle', fontsize = 18)

In [6]:
# I need to get an example with real data.

# Make an instance and perform the imputation
imputer = MissForest()
train_df_imputed = imputer.fit_transform(train_df)

ValueError: could not convert string to float: 'RL'

In [7]:
train_df['RL']

KeyError: 'RL'

In [13]:
#dropping NA to avoid issues with lgb fit.
train_df['SalePrice'].dropna(inplace = True)

print(len(train_df))


1460


In [39]:
# getting ready x and y variables
x = train_df.drop(['Id','SalePrice'],axis=1)
y = train_df.SalePrice

# train test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.33, random_state = 42)

In [41]:
from sklearn.datasets import load_boston

boston = load_boston()
x, y = boston.data, boston.target
x_df = pd.DataFrame(x, columns= boston.feature_names)
x_train, x_test, y_train, y_test = train_test_split(x_df, y, test_size=0.15) 

/Users/hectorcozar/Documents/GitHub/house_prizes_kaggle_python/venv/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternati

In [42]:
# defining parameters 
params = {
    'task': 'train', 
    'boosting': 'gbdt',
    'objective': 'regression',
    'num_leaves': 10,
    'learnnig_rage': 0.05,
    'metric': {'l2','l1'},
    'verbose': -1
}

In [43]:
# laoding data
lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_test, y_test, reference=lgb_train)
 

In [44]:
# fitting the model
model = lgb.train(params,
                 train_set=lgb_train,
                 valid_sets=lgb_eval,
                 early_stopping_rounds=30)

/Users/hectorcozar/Documents/GitHub/house_prizes_kaggle_python/venv/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Unknown parameter: learnnig_rage
[1]	valid_0's l2: 89.8079	valid_0's l1: 6.71541
Training until validation scores don't improve for 30 rounds
[2]	valid_0's l2: 77.5728	valid_0's l1: 6.20691
[3]	valid_0's l2: 67.529	valid_0's l1: 5.74798
[4]	valid_0's l2: 59.2784	valid_0's l1: 5.3473
[5]	valid_0's l2: 52.3859	valid_0's l1: 4.98802
[6]	valid_0's l2: 46.6775	valid_0's l1: 4.67455
[7]	valid_0's l2: 42.038	valid_0's l1: 4.4116
[8]	valid_0's l2: 37.9651	valid_0's l1: 4.17738
[9]	valid_0's l2: 34.9744	valid_0's l1: 3.98992
[10]	valid_0's l2: 32.0874	valid_0's l1: 3.82012
[11]	valid_0's l2: 29.7616	valid_0's l1: 3.66791
[12]	valid_0's l2: 27.5808	valid_0's l1: 3.54418
[13]	valid_0's l2: 25.6489	valid_0's l1: 3.39711
[14]	valid_0's l2: 24.1938	valid_0's l1: 3.30084
[15]	valid_0's l2: 22.8786	valid_0's l1: 3.22715
[16]	valid_0's l2: 21.6985	valid_0's l1: 3.13521
[17]	valid_0's l2: 20.7678	valid_0's l1: 3.07105
[18]	valid_0's l2: 19.7471	valid_0's l1: 2.98498
[19]	valid_0's l

In [45]:
# prediction
y_pred = model.predict(x_test)

# accuracy check
mse = mean_squared_error(y_test, y_pred)
rmse = mse**(0.5)
print("MSE: %.2f" % mse)
print("RMSE: %.2f" % rmse) 

NameError: name 'mean_squared_error' is not defined

In [32]:
#setting up the model
model = lgb.LGBMClassifier(learning_rate = 0.09, max_depth = -5, random_state = 42)
model.fit(x_train, y_train, eval_set = [(x_test, y_test), (x_train, y_train)],
          verbose = 20, eval_metric = 'logloss')



ValueError: y contains previously unseen labels: [34900, 52500, 55993, 64500, 68500, 72500, 73000, 75500, 80500, 83500, 84000, 84900, 85000, 89000, 89471, 91300, 94500, 101800, 103200, 108500, 109000, 116900, 117000, 118858, 121500, 128200, 128950, 131400, 134450, 135960, 139400, 139900, 142953, 143750, 145250, 145900, 147400, 147500, 149350, 151400, 154300, 155835, 155900, 156500, 158500, 159895, 162500, 164700, 164900, 165150, 165400, 167240, 167900, 169990, 171500, 174500, 174900, 178400, 178900, 179540, 183500, 184750, 186500, 198900, 200100, 200500, 204900, 205950, 206900, 207000, 209500, 212900, 213250, 213490, 214500, 216000, 216500, 216837, 223000, 226700, 227680, 230500, 232600, 243000, 244400, 245000, 250580, 252000, 254900, 255500, 256300, 260400, 262280, 264132, 269500, 274000, 274300, 274725, 274970, 276000, 277500, 281000, 294000, 295000, 295493, 299800, 303477, 305900, 307000, 309000, 311872, 312500, 314813, 318000, 319900, 325624, 326000, 328900, 336000, 337000, 341000,

In [6]:
#Accuracy
print('Training accuracy {:.4f}'.format(model.score(x_train,y_train)))
print('Testing accuracy {:.4f}'.format(model.score(x_test,y_test)))

In [ ]:
#Relevant variables
lgb.plot_importance(model)

In [ ]:
# Learning curve of the model
lgb.plot_metric(model)


In [ ]:
# plot tree of the model
lgb.plot_tree(model,figsize=(30,40))

In [ ]:
# confusion matrix
metrics.plot_confusion_matrix(model,x_test,y_test,cmap='Blues_r')

In [ ]:
# metrics
print(metrics.classification_report(y_test,model.predict(x_test)))

In [ ]:
#predict and getting data out from h2o
predictions = 

In [15]:
#merging predictions with the full data set
submission = pd.merge(test_df['Id'],
                          predictions,
                          left_index = True,
                          right_index = True)
submission.rename(columns = {'predict': 'SalePrice'}, inplace = True)

In [16]:
# save to csv
submission.to_csv('../data/submission_' + best_model_name + '.csv',
                  header = True, index = False)

# 00. Score of 0.12868
# 01. Change from 25 max models to 1000. Score of 0.12792
# 02. Sorting leaderboard by rmse as the main metric on h2o training. Score of 0.12792 made the same.
# 02. Gonna make models for 30 min instead doing 1000 models. No improvement 0.13452
# 02. We're going with 10.000 models. Score of 0.12970... no improving
# 03. Trying with xgboost